<a href="https://colab.research.google.com/github/gabriel1628/Meeting-Scheduling-Assistant/blob/main/zero_shot_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [5]:
from transformers import pipeline
from datasets import load_from_disk
import os

In [3]:
ds = load_from_disk("/content/drive/MyDrive/Colab Notebooks/AI Meeting Assistant/data")
ds

Dataset({
    features: ['text', 'label'],
    num_rows: 264
})

In [6]:
# Define the cache directory in your Google Drive
cache_dir = '/content/drive/My Drive/huggingface_models'

# Create the directory if it doesn't exist
os.makedirs(cache_dir, exist_ok=True)

# Set the cache directory for the transformers library
os.environ['TRANSFORMERS_CACHE'] = cache_dir
os.environ['HF_HOME'] = cache_dir
os.environ['HF_HUB_CACHE'] = cache_dir

In [7]:
pipe = pipeline(model="facebook/bart-large-mnli", device=0)
pipe

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [63]:
import datasets
ds = ds.cast_column("label", datasets.ClassLabel(names=["meeting scheduling", "other"]))
ds.features

Casting the dataset:   0%|          | 0/264 [00:00<?, ? examples/s]

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['meeting scheduling', 'other'], id=None),
 'label_text': Value(dtype='string', id=None)}

In [64]:
candidate_labels = ds.features["label"].names
candidate_labels

['meeting scheduling', 'other']

In [65]:
%%time
zeroshot_preds = pipe(ds["text"], batch_size=16, candidate_labels=candidate_labels)

CPU times: user 3.45 s, sys: 3.63 ms, total: 3.46 s
Wall time: 3.46 s


In [66]:
zeroshot_preds[:5]

[{'sequence': 'Arrange a meeting with Florian this week or next week in the afternoon',
  'labels': ['meeting scheduling', 'other'],
  'scores': [0.8793452978134155, 0.12065468728542328]},
 {'sequence': 'Schedule a meeting with Marc tomorrow morning or Thursday afternoon',
  'labels': ['meeting scheduling', 'other'],
  'scores': [0.9503199458122253, 0.049680035561323166]},
 {'sequence': 'Set up a call with the sales director Monday or Tuesday in the early afternoon',
  'labels': ['meeting scheduling', 'other'],
  'scores': [0.6569020748138428, 0.3430979549884796]},
 {'sequence': 'Arrange an interview with the marketing team Wednesday or Thursday morning',
  'labels': ['meeting scheduling', 'other'],
  'scores': [0.8329837322235107, 0.16701629757881165]},
 {'sequence': 'Organize a conference with stakeholders Friday or next Monday in the late morning',
  'labels': ['meeting scheduling', 'other'],
  'scores': [0.8079595565795898, 0.19204042851924896]}]

In [67]:
predicted_labels = [pred["labels"][0] for pred in zeroshot_preds]
predicted_labels[:5]

['meeting scheduling',
 'meeting scheduling',
 'meeting scheduling',
 'meeting scheduling',
 'meeting scheduling']

In [68]:
# create a new column with the numeric label verbalised as label_text (e.g. "positive" instead of "0")
label_map = {
    i: label_text
    for i, label_text in enumerate(ds.features["label"].names)
}

def add_label_text(example):
    example["label_text"] = label_map[example["label"]]
    return example

ds = ds.map(add_label_text)

print(ds)

Map:   0%|          | 0/264 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label', 'label_text'],
    num_rows: 264
})


In [69]:
true_labels = ds["label_text"]
true_labels[:5]

['meeting scheduling',
 'meeting scheduling',
 'meeting scheduling',
 'meeting scheduling',
 'meeting scheduling']

In [70]:
from sklearn.metrics import classification_report

print(classification_report(true_labels, predicted_labels))

                    precision    recall  f1-score   support

meeting scheduling       0.97      0.99      0.98       162
             other       0.98      0.95      0.97       102

          accuracy                           0.97       264
         macro avg       0.97      0.97      0.97       264
      weighted avg       0.97      0.97      0.97       264



In [71]:
misclassified = []
for i in range(len(ds)):
  if predicted_labels[i] != true_labels[i]:
    print(f"sentence : {ds[i]['text']}")
    print(f"predicted label : {predicted_labels[i]}")
    print(f"true label : {true_labels[i]}")
    print(f"prediction scores : {zeroshot_preds[i]['scores']}")
    print()
    misclassified.append(i)

sentence : Arrange a call with the IT department for Tuesday afternoon
predicted label : other
true label : meeting scheduling
prediction scores : [0.6119756102561951, 0.38802438974380493]

sentence : Schedule a video conference with the remote team on Monday morning
predicted label : other
true label : meeting scheduling
prediction scores : [0.6690781116485596, 0.3309219479560852]

sentence : The CEO will be visiting our office next week.
predicted label : meeting scheduling
true label : other
prediction scores : [0.6710972189903259, 0.3289027512073517]

sentence : I'd like to schedule a meeting with the design team.
predicted label : meeting scheduling
true label : other
prediction scores : [0.93917316198349, 0.0608268678188324]

sentence : The new smartphone is expected to be released next month.
predicted label : meeting scheduling
true label : other
prediction scores : [0.5248702764511108, 0.4751296937465668]

sentence : The city is hosting a marathon next month.
predicted label :

In [72]:
print(misclassified)

[27, 30, 166, 167, 174, 216, 252]


In [73]:
print(f"{len(misclassified)} incorrect predictions out of {len(ds)}")

7 incorrect predictions out of 264


In [75]:
ds[misclassified]["text"]

['Arrange a call with the IT department for Tuesday afternoon',
 'Schedule a video conference with the remote team on Monday morning',
 'The CEO will be visiting our office next week.',
 "I'd like to schedule a meeting with the design team.",
 'The new smartphone is expected to be released next month.',
 'The city is hosting a marathon next month.',
 'The park is hosting a free concert series next month.']